* Image Augmentation for CV
* Classifying objects to images
* Localizing an object in images
* Segmentation classes in images
* Finding facial points
* Recognizing faces

### Purpose
* Increase the datasets so that training is robust.
* Augmentation needs domain knowledge
* Uses OpenCV

<img src="https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/more_images/LocalizationDetection.png">

* Use regression to find co-ordinates of the localized image

<img src="https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/more_images/LocalizationRegression2.png">

### Object Localization

* Finding position

In [7]:
DATA_DIR = r'C:\Users\ZekeLabs\Downloads\object-detection-crowdai'

In [4]:
import pandas as pd

In [8]:
labels = pd.read_csv(DATA_DIR + '\labels.csv', usecols=[0,1,2,3,4,5])

In [9]:
labels.head()

,xmin,xmax,ymin,ymax,Frame,Label
0,785,533,905,644,1479498371963069978.jpg,Car
1,89,551,291,680,1479498371963069978.jpg,Car
2,268,546,383,650,1479498371963069978.jpg,Car
3,455,522,548,615,1479498371963069978.jpg,Truck
4,548,522,625,605,1479498371963069978.jpg,Truck


In [11]:
labels = labels[labels.Label == 'Truck']

In [12]:
labels.shape

(3819, 6)

In [13]:
labels.head()

,xmin,xmax,ymin,ymax,Frame,Label
3,455,522,548,615,1479498371963069978.jpg,Truck
4,548,522,625,605,1479498371963069978.jpg,Truck
12,368,528,468,624,1479498372942264998.jpg,Truck
13,472,528,550,611,1479498372942264998.jpg,Truck
18,417,534,509,640,1479498373962951201.jpg,Truck


In [14]:

image_list = ['1479498416965787036.jpg',
              '1479498541974858765.jpg']

In [16]:
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
plt.figure(figsize=(15,15))

<img src="https://www.safaribooksonline.com/library/view/deep-learning-for/9781788295628/assets/63fb2c41-8e83-49c5-ad3a-fee59e8a178b.png">

In [19]:
def calculate_iou(gt_bb, pred_bb):
    '''
    :param gt_bb: ground truth bounding box
    :param pred_bb: predicted bounding box
    '''
    gt_bb = tf.stack([
        gt_bb[:, :, :, :, 0] - gt_bb[:, :, :, :, 2] / 2.0,
        gt_bb[:, :, :, :, 1] - gt_bb[:, :, :, :, 3] / 2.0,
        gt_bb[:, :, :, :, 0] + gt_bb[:, :, :, :, 2] / 2.0,
        gt_bb[:, :, :, :, 1] + gt_bb[:, :, :, :, 3] / 2.0])
    gt_bb = tf.transpose(gt_bb, [1, 2, 3, 4, 0])
    pred_bb = tf.stack([
        pred_bb[:, :, :, :, 0] - pred_bb[:, :, :, :, 2] / 2.0,
        pred_bb[:, :, :, :, 1] - pred_bb[:, :, :, :, 3] / 2.0,
        pred_bb[:, :, :, :, 0] + pred_bb[:, :, :, :, 2] / 2.0,
        pred_bb[:, :, :, :, 1] + pred_bb[:, :, :, :, 3] / 2.0])
    pred_bb = tf.transpose(pred_bb, [1, 2, 3, 4, 0])
    area = tf.maximum(
        0.0,
        tf.minimum(gt_bb[:, :, :, :, 2:], pred_bb[:, :, :, :, 2:]) -
        tf.maximum(gt_bb[:, :, :, :, :2], pred_bb[:, :, :, :, :2]))
    intersection_area= area[:, :, :, :, 0] * area[:, :, :, :, 1]
    gt_bb_area = (gt_bb[:, :, :, :, 2] - gt_bb[:, :, :, :, 0]) * \
                 (gt_bb[:, :, :, :, 3] - gt_bb[:, :, :, :, 1])
    pred_bb_area = (pred_bb[:, :, :, :, 2] - pred_bb[:, :, :, :, 0]) * \
                   (pred_bb[:, :, :, :, 3] - pred_bb[:, :, :, :, 1])
    union_area = tf.maximum(gt_bb_area + pred_bb_area - intersection_area, 1e-10)
    iou = tf.clip_by_value(intersection_area / union_area, 0.0, 1.0)
    return iou

### Mean average Precision

In [ ]:
* mAP used in evaluation detection algorithms
* Value ranges from 0 to 100, higher the value better it is